In [62]:
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
import os
import cv2 
from PIL import Image
from tqdm import tnrange, tqdm_notebook
import numpy as np
from numpy import moveaxis

## First step - extracting training data from the timelapse
* This involves trimming the timelapse into four parts 
* Then extracting all the frames from the trimmed videos to be converted to numpy arrays

In [21]:
def trim_video(source_path, start_time, end_time, target_path):
    ffmpeg_extract_subclip(source_path, start_time, end_time, targetname=target_path)

In [29]:
trim_video(source_path = "c_elegans/c_elegans_timelapse.MP4", start_time = 32, end_time = 82, target_path = "split_labels/gastrula.mp4")
trim_video(source_path = "c_elegans/c_elegans_timelapse.MP4", start_time = 116, end_time = 126, target_path = "split_labels/comma.mp4")
trim_video(source_path = "c_elegans/c_elegans_timelapse.MP4", start_time = 140, end_time = 187, target_path = "split_labels/fold.mp4")
trim_video(source_path = "c_elegans/c_elegans_timelapse.MP4", start_time = 224, end_time = 240, target_path = "split_labels/l1.mp4")


Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful


In [49]:
def videos_to_frames(video_path, new_folder_path):
    os.mkdir(new_folder_path)
    vidObj = cv2.VideoCapture(video_path)   
    success = 1
    count = 0
    while success: 
        success, image = vidObj.read() 
        fmt_name = new_folder_path + "/" + str(count)+ ".jpg"
        cv2.imwrite(fmt_name, image) 
        count += 1
    print ("saved ", count-1, " frames at ", new_folder_path)

In [71]:
videos_to_frames("split_labels/gastrula.mp4", new_folder_path = "frames/gastrula")
videos_to_frames("split_labels/comma.mp4", new_folder_path = "frames/comma")
videos_to_frames("split_labels/fold.mp4", new_folder_path = "frames/fold")
videos_to_frames("split_labels/l1.mp4", new_folder_path = "frames/l1")

saved  750  frames at  frames/gastrula
saved  150  frames at  frames/comma
saved  705  frames at  frames/fold
saved  237  frames at  frames/l1


## Generating training data from saved frames
* img_to_arr() takes in the path of an image and returns a numpy array of the image, moveaxis() is optional depending on whether we want to feed it to a CNN or to simply just show it.
* Keep moveaxis = False when you want to plot and view it, or moveaxis = True when it needs to be used for training data


In [74]:
def img_to_arr(im_path, move_axis = True):
    try:
        img = Image.open(im_path)
    except:
        return
    x = np.array(img)
    if move_axis == True:
        x = moveaxis(x, 2, 0)
    return x

In [76]:
def generate_train_data(source_folder, save_path, filename,  label = 0):
    save_path = save_path + filename
    final_stack = []
    c = 0
    for i in tqdm_notebook(os.listdir(source_folder)):   
        
        im_path = source_folder + "/" + i
        im_arr = img_to_arr(im_path)
        stack_element = np.array([im_arr, label])
        final_stack.append(stack_element)
    
    
    final_stack = np.array(final_stack)
    print (final_stack.shape[0], " instances in ", filename)
    fmt_name = save_path
    np.save(fmt_name, final_stack)
    print ("saved -- ", fmt_name)

## Save all the frames with labels from the folders as numpy arrays
* These portable numpy arrays can be wrapped with the TensorDataset wrapper for training the CNN



In [80]:
generate_train_data("frames/gastrula/", "numpy/", "gastrula.npy",  label = 0)
generate_train_data("frames/comma/", "numpy/", "comma.npy",  label = 1)
generate_train_data("frames/fold/", "numpy/", "fold.npy",  label = 2)
generate_train_data("frames/l1/", "numpy/", "l1.npy",  label = 3)


751  instances in  gastrula.npy
saved --  numpy/gastrula.npy



151  instances in  comma.npy
saved --  numpy/comma.npy



706  instances in  fold.npy
saved --  numpy/fold.npy



238  instances in  l1.npy
saved --  numpy/l1.npy
